In [8]:
import lightgbm
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from joblib import dump
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from joblib import dump
import optuna
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import joblib
import optuna.visualization as vis
from sklearn.model_selection import cross_val_score, KFold
import warnings
warnings.filterwarnings("ignore")
import optuna



In [9]:
# データの読み込み
train_data = pd.read_csv('../data_processed/train_data_processed.csv')
test_data = pd.read_csv('../data_processed/test_data_processed.csv')
submit_data = pd.read_csv('../data/test.csv')

X = train_data.drop(['attendance'], axis=1)
y = train_data['attendance']


# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# 目的関数の定義
def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 16),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.00001, 0.1),
        'iterations': trial.suggest_int('iterations', 50, 2000),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 20),
        'border_count': trial.suggest_categorical('border_count', [32, 64, 128, 256, 512]),
        'thread_count': 4
    }
    
    rmse_scores = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model = CatBoostRegressor(**params, random_seed=42, verbose=0)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, 
                  cat_features=['venue', 'weather', 'day_of_week', 'is_holiday', 'season'])
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        rmse_scores.append(rmse)
    
    avg_rmse = np.mean(rmse_scores)
    return avg_rmse

# 最適化の実行
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
study.optimize(objective, n_trials=200, timeout=7200)

# 最適なパラメータの出力
print('Best trial:', study.best_trial.params)


[I 2023-06-29 04:12:17,291] A new study created in memory with name: no-name-2c677e18-11e9-4d68-a0e2-e04e1d2e218b
[I 2023-06-29 04:12:19,987] Trial 0 finished with value: 9134.31638584718 and parameters: {'depth': 9, 'learning_rate': 0.0012725990125870446, 'iterations': 61, 'l2_leaf_reg': 12, 'border_count': 128}. Best is trial 0 with value: 9134.31638584718.
[I 2023-06-29 04:12:45,287] Trial 1 finished with value: 4190.234472453248 and parameters: {'depth': 6, 'learning_rate': 0.02662896933992949, 'iterations': 1263, 'l2_leaf_reg': 13, 'border_count': 64}. Best is trial 1 with value: 4190.234472453248.
[I 2023-06-29 04:12:50,874] Trial 2 finished with value: 9390.102876402063 and parameters: {'depth': 8, 'learning_rate': 7.910624170919946e-05, 'iterations': 210, 'l2_leaf_reg': 11, 'border_count': 64}. Best is trial 1 with value: 4190.234472453248.
[I 2023-06-29 04:13:15,469] Trial 3 finished with value: 7124.561059916154 and parameters: {'depth': 9, 'learning_rate': 0.0014790807101296

In [ ]:
# 学習曲線の描画
vis.plot_optimization_history(study)

In [ ]:
# ハイパーパラメータ探索空間の視覚化
vis.plot_parallel_coordinate(study)

In [ ]:
# ハイパーパラメータ探索空間の視覚化
vis.plot_parallel_coordinate(study)

In [12]:
# 最適なモデルを取得
best_params = study.best_trial.params
best_model = CatBoostRegressor(**best_params, random_seed=42, verbose=0)
best_model.fit(X, y, cat_features=['venue', 'weather', 'day_of_week', 'is_holiday', 'season'])

# モデルを保存
best_model.save_model('../models/CatBoost(optuna)')